In [21]:
from qbraid.devices.utils import get_devices
from qbraid.devices.utils import device_wrapper
from braket.devices import LocalSimulator as BraketSimulator
from cirq.devices.device import Device as CirqDevice
from qbraid.devices.utils import SUPPORTED_VENDORS

In [17]:
get_devices()

Software Vendor    Device Provider    Device Name
-----------------  -----------------  -----------------
AWS                AWS                SV1
AWS                AWS                DM1
AWS                AWS                TN1
AWS                AWS                braket_sv
AWS                AWS                braket_dm
AWS                AWS                default
AWS                D-Wave             DW_2000Q_6
AWS                D-Wave             Advantage_system1
AWS                IonQ               ionQdevice
AWS                Rigetti            Aspen-9
Google             Google             Sycamore
Google             Google             Sycamore23
IBM                IBM                AerSimulator
IBM                IBM                PulseSimulator


In [18]:
def device_wrapper_input(vendor):
    input_list = []
    for provider in SUPPORTED_VENDORS[vendor]:
        for device in SUPPORTED_VENDORS[vendor][provider]:
            data = (device, provider, vendor)
            input_list.append(data)
    return input_list

In [33]:
input1, input2 = device_wrapper_input('IBM')

In [37]:
qbraid_device = device_wrapper(*input1)

In [38]:
type(qbraid_device)

qbraid.devices.ibm.device.IBMDeviceWrapper

In [40]:
qiskit_device = qbraid_device.vendor_device_obj

In [41]:
type(qiskit_device)

abc.ABCMeta